In [4]:
pip install thermo

  Using cached thermo-0.2.21-py3-none-any.whl (7.4 MB)
  Using cached fluids-1.0.21-py3-none-any.whl (2.3 MB)
  Using cached chemicals-1.1.1-py3-none-any.whl (23.3 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.constants import calorie, R, atm
import os
import thermo

In [12]:
os.getcwd()

'/root/Molecular_distillation_simulator'

In [6]:
def nrtl(T, alpha1, alpha2, g1, g2, x):
    '''
    NRTLによる活量係数の計算

    Parameters
    ----------
    alpha : ndarray(n,n)
        Array of NRTL nonrandomness parameters. n = the number of 
        components in the system.
    tau : ndarray(n,n)
        Array of NRTL tau parameters. tau[i,i] should be set to 0.
    t : float
        Temperature (K)
    x : ndarray(n,)
        Mole fraction of each component

    Returns
    -------
    gamma : ndarray(n,)
        Activity coefficient of each component    
    '''
    alpha = np.asarray([alpha1, alpha2])
    g = np.asarray([[0, g1],
                  [g2, 0]])
    
    tau = g / (R*T)
    G = np.exp(-alpha*tau)
    ncomp = x.shape[0]
    gamma = np.zeros_like(x)
    summ = 0
    
    for i in range(ncomp):
        summ = 0
        for j in range(ncomp):
            summ += x[j]*G[i,j]/np.sum(G[:,j]*x)*(tau[i,j] - (np.sum(x*tau[:,j]*G[:,j])/np.sum(G[:,j]*x)))
        gamma[i] = np.sum(tau[:,i]*G[:,i]*x)/np.sum(G[:,i]*x) + summ
        
        return np.exp(gamma)
    
def opt_test():
    T = 138 + 273.15
    g_ini = np.asarray([[0, 1.0],
                    [1.0, 0]])
    tau_ini = g_ini / (R*T)
    alpha_ini = 0.2
    x_ini = np.asarray([0.9898662364, 0.0101337636])
    gamma_ini = nrtl(alpha_ini, tau_ini, x_ini)
    print(gamma_ini)
    
def equilibrium(pi, x, y, p):
    '''
    低圧の気液平衡関係式から活量係数を求める
    '''
    gamma_lab = (pi * y) / (p * x)
    return gamma_lab

def lab_read(pi):
    '''
    実験データの読み取り
    3atm, 5atm, 8atm, 11.2atmそれぞれの圧力条件でのデータ
    '''
    df_lab = pd.read_csv("/root/Molecular_distillation_simulator/table.csv").drop(index=[0, 1]).reset_index().drop(columns="index")
    
    df_atm3 = pd.DataFrame(df_lab.iloc[0:10, 0:4]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                         'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1)
    # print(df_atm3)
    df_atm5 = pd.DataFrame(df_lab.iloc[0:11, 4:8]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                         'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1)
    # print(df_atm5)
    df_atm8 = pd.DataFrame(df_lab.iloc[12:24, 0:4]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                          'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1).reset_index().drop(columns="index")
    # print(df_atm8)
    df_atm11 = pd.DataFrame(df_lab.iloc[12:23, 4:8]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                          'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1).reset_index().drop(columns="index")
    # print(df_atm11)
    if pi == 3:
        return df_atm3
    elif pi == 5:
        return df_atm5
    elif pi == 8:
        return df_atm8
    else:
        return df_atm11

In [41]:
#ラボデータ、初期条件
df_lab = lab_read(3)
pi = 3 * atm
t = np.float64(df_lab.iloc[0, 3])
T = t + 273.15

#蒸気圧 ※今後は厳密計算を行う必要がある
from thermo.vapor_pressure import *
p1 = VaporPressure(CASRN="67-56-1").calculate(T, "ANTOINE_POLING")
p2 = VaporPressure(CASRN="7732-18-5").calculate(T, "ANTOINE_POLING")

#ラボデータを用いて計算した活量係数： gamma_lab
gamma_lab1 = equilibrium(pi, np.float64(df_lab.iloc[0, 1]), np.float64(df_lab.iloc[0, 2]), p1)
gamma_lab2 = equilibrium(pi, 1 - np.float64(df_lab.iloc[0, 1]), 1 - np.float64(df_lab.iloc[0, 2]), p2)

#NRTLから計算した活量係数：　gamma_cal
x_ini = np.asarray([0.9, 0.1])
#alpha_ini = np.asarray([0.2, 0.2])
#g_ini = np.asarray([[0, 2022],[1000, 0]])
alpha_ini1, alpha_ini2, g_ini1, g_ini2 = 0.2, 0.2, 100, 100 # g_iniは適当な初期値

gamma_cal = nrtl(T, alpha_ini1, alpha_ini2, g_ini1, g_ini2, x_ini)

#誤差の関数
#E = np.sqrt((gamma_lab1 - gamma_cal[0])**2) + np.sqrt((gamma_lab2 - gamma_cal[1])**2)
def E(X):
    gamma_lab1 = equilibrium(pi, np.float64(df_lab.iloc[0, 1]), np.float64(df_lab.iloc[0, 2]), p1)
    gamma_lab2 = equilibrium(pi, 1 - np.float64(df_lab.iloc[0, 1]), 1 - np.float64(df_lab.iloc[0, 2]), p2)
    gamma_cal = nrtl(T, X[0], X[1], X[2], X[3], x_ini)
    np.sqrt((gamma_lab1 - gamma_cal[0])**2) + np.sqrt((gamma_lab2 - gamma_cal[1])**2)
    return E

#Eのoptimize
x0 = [0.2, 0.2, 100, 100]
bounds_x1 = (0.20, 0.47)
bounds_x2 = (0.20, 0.47)
bounds_x3 = (-100, 100)
bounds_x4 = (-100, 100)
bound = (bounds_x1, bounds_x2, bounds_x3, bounds_x4)
result = minimize(E, x0, method="Nelder-Mead", bounds=bound)

TypeError: float() argument must be a string or a number, not 'function'

((0.2, 0.47), (0.2, 0.47), (-10000, 10000), (-10000, 10000))